In [1]:
import torch
from torch import optim,nn
import torch.nn.functional as F
from torch.autograd import Variable
import gc
import numpy as np
import time
import os
import random
import pickle
import argparse
from functions import get_dates,calculate
from sklearn.metrics import roc_auc_score as AUC
from sklearn.metrics import average_precision_score as AP

In [2]:
from models.gru_bidirectional import GRU
from models.retain_bidirectional import RETAIN
from models.retain_ex import RETAIN_EX

In [3]:
ver_list = ['retain','ex','gru']
task_list = ['I50','H26']
time_list = [0,1]
hid_list = [64,128,256]
lr_list = [0.01, 0.001, 0.0001]
epoch_list = np.arange(30)+1

In [ ]:
ver='gru'
lr = 0.01
hid = 128
task = 'I50'
if ver=='gru':
    from models.gru_bidirectional import GRU
with open('data/%s/val.pckl'%task,'rb') as f:
    val_data = pickle.load(f)
model = GRU(hid,hid,1,cuda_flag=True)
model.load_state_dict(torch.load('experiments/H26/saved_weights/lr_0.001/retain_time_interpret/epoch/10_epochs.pth'))
# model.load_state_dict(torch.load('experiments2/I50/gru_128_0.0100/saved_weights/2_cuda.pckl'))
model.cuda()

In [14]:
print('task\tver\ttime\thid\tlr\tepoch\tauc\tap')
for task in task_list:
    with open('data/%s/val.pckl'%task,'rb') as f:
        val_data = pickle.load(f)
#     for hid in [128]:
    for hid in hid_list:
            for time_fn in time_list:
                for ver in ver_list:
                        if (time_fn==1)&(ver!='ex'):
                            continue
                        else:
                            if ver=='gru':
                                model = GRU(hid,hid,1,cuda_flag=True)
                            elif ver=='retain':
                                model = RETAIN(hid,hid,1,cuda_flag=True)
                            elif (ver=='ex'):
                                model = RETAIN_EX(hid,hid,1,cuda_flag=True,bidirectional=True,time_ver=time_fn)
                            model.cuda()    
                            for lr in lr_list:
            #                 for lr in ['0.001']:
                                for epoch in epoch_list[:10]:
            #                     for epoch in [2]:
                                    if ver=='ex':
                                        weight_name = 'experiments2/%s/%s-%d_%d_%s/saved_weights/%d_cpu.pckl'%(task,ver,time_fn,hid,str(lr),epoch)
                                    else:
                                        weight_name = 'experiments2/%s/%s_%d_%s/saved_weights/%d_cpu.pckl'%(task,ver,hid,lr,epoch)
                                    try:
                                        model.load_state_dict(torch.load(weight_name))
                                        correct_list = []
                                        score_list = []
                                        for (X,y) in val_data:
                                            outputs,targets = calculate(X,y,model,ver=ver,cuda_flag=True,time_fn=time_fn)
                                            correct_list.extend(y)
                                            score_list.extend(outputs.squeeze().data.tolist())
                                        auc = AUC(y_true=correct_list,y_score=score_list)
                                        ap = AP(y_true=correct_list,y_score=score_list)
                                        print('%s\t%s\t%d\t%d\t%s\t%d\t%1.3f\t%1.3f'%(task,ver,time_fn,hid,str(lr),epoch,auc,ap))
                                    except FileNotFoundError:
            #                             print('file not found')
                                        continue

task	ver	time	hid	lr	epoch	auc	ap
I50	ex	1	64	0.001	1	0.873	0.567
I50	ex	1	64	0.001	2	0.928	0.720
I50	ex	1	64	0.001	3	0.908	0.704
I50	ex	1	64	0.001	4	0.912	0.703
I50	ex	1	64	0.001	5	0.915	0.706
I50	retain	0	128	0.001	1	0.821	0.468
I50	retain	0	128	0.001	2	0.860	0.553
I50	retain	0	128	0.001	3	0.882	0.620
I50	retain	0	128	0.001	4	0.886	0.653
I50	retain	0	128	0.001	5	0.862	0.611
I50	ex	0	128	0.001	1	0.901	0.631
I50	ex	0	128	0.001	2	0.932	0.732
I50	ex	0	128	0.001	3	0.931	0.769
I50	ex	0	128	0.001	4	0.916	0.723
I50	ex	0	128	0.001	5	0.919	0.715
I50	gru	0	128	0.001	1	0.815	0.479
I50	gru	0	128	0.001	2	0.873	0.582
I50	gru	0	128	0.001	3	0.892	0.621
I50	gru	0	128	0.001	4	0.883	0.624
I50	gru	0	128	0.001	5	0.885	0.628
I50	ex	1	128	0.001	1	0.897	0.626
I50	ex	1	128	0.001	2	0.936	0.750
I50	ex	1	128	0.001	3	0.928	0.708
I50	ex	1	128	0.001	4	0.931	0.735
I50	ex	1	128	0.001	5	0.922	0.722
I50	ex	1	256	0.001	1	0.906	0.597
I50	ex	1	256	0.001	2	0.934	0.681
I50	ex	1	256	0.001	3	0.929	0.686
I50	ex	1	256	0.001	4	0

In [10]:
weight_name

'experiments2/H26/ex-1_128_0.001/saved_weights/3_cpu.pckl'